# Importing llamas_pyjamas modules

In [ ]:
import os
import sys
import numpy as np
import pickle
import ray
import pkg_resources
import glob
import traceback
from pathlib import Path
from config import BASE_DIR, OUTPUT_DIR, DATA_DIR


# Get package root and add to path before other imports
package_root = Path().absolute().parent
sys.path.append(str(package_root))
sys.path.append(BASE_DIR+'/')


ray.init(ignore_reinit_error=True)
from llamas_pyjamas.Trace.traceLlamasMulti import main # type: ignore
from llamas_pyjamas.Extract.extractLlamas import ExtractLlamas, save_extractions, load_extractions # type: ignore
from llamas_pyjamas.Image.WhiteLight import WhiteLight, WhiteLightFits
from llamas_pyjamas.QA.llamasQA import plot_ds9



In [ ]:
# Get absolute path to llamas_pyjamas package
package_path = pkg_resources.resource_filename('llamas_pyjamas', '')
package_root = os.path.dirname(package_path)

print(f"Package path: {package_path}")
print(f"Package root: {package_root}")

In [ ]:
# Configure Ray runtime environment
runtime_env = {
    "py_modules": [package_root],
    "env_vars": {"PYTHONPATH": f"{package_root}:{os.environ.get('PYTHONPATH', '')}"},
    "excludes": [
        str(Path(DATA_DIR) / "**"),  # Exclude DATA_DIR and all subdirectories
        "**/*.fits",                 # Exclude all FITS files anywhere
        "**/*.pkl",                  # Exclude all pickle files anywhere
        "**/.git/**",               # Exclude git directory
    ]
}

# Initialize Ray
ray.shutdown()
ray.init(runtime_env=runtime_env)

Running the trace finding algorithm on all of the HDU extensions

In [ ]:
#fitsfile = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/LLAMAS_2024-11-20T13_15_33.000_mef.fits'
filename = os.path.join(DATA_DIR, 'LLAMAS_2024-11-20T13_15_33.000_mef.fits') 
main(filename)

# or run on the command line with the fitsfile as an input argument


## To generate a WhiteLight image run the following command on the terminal:

In [ ]:

#make sure you are in the llamas_pyjamas directory for this
#This is currently set up to do the full extraction and trace on a single file, the fast extraction needs updating to work
python3 __main__.py raw_fits_file_path

## To reproduce the Look Up Table (LUT) for the combs and fibre positions

In [ ]:
from llamas_pyjamas.Utils import dump_LUT

channel = 'green' #re-run this for each color

calibration_fits_file = 'path/to/raw/calibration/file'

hdu = fits.open(calibration_fits_file)
trace = TraceLlamas(calibration_fits_file, find_LUT=True)

dump_LUT(channel, hdu, trace)

## To flip fibre position ordering in the LUT based on the colour and side

In [ ]:
from llamas_pyjamas.Utils import flip_positions

flipped = {"greenA":True, "greenB":False, "blueA": False, "blueB":True, "redA":True, "redB":False}

flip_positions()